In [1]:
# run dinucleosomes with openmm
# start from running one hexamer, and finally try if we can run multiple hexamers
# load the packages and set the environment
# we also aim to generalize this code to other chromatin multimers

import sys
import numpy as np
import pandas as pd
import simtk.openmm
import pdbfixer
import os
import glob
import shutil

# Shuming's working environment on desktop
open3SPN2_HOME = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/open3spn2'
calpha_HOME = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM'

# Shuming's working environment on laptop
#open3SPN2_HOME = '/Users/smliu/Documents/Projects/CA_SBM_3SPN2C_OPENMM/open3spn2'
#calpha_HOME = '/Users/smliu/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM'
sys.path.insert(0, open3SPN2_HOME)
sys.path.insert(0, calpha_HOME)

import open3SPN2
import ffCalpha
import ffCalphaExt


# Build the CG model

In [2]:
# the source code of fixPDB from open3spn2 with some modifications to accelerate loading PDB files
# basically we can stop finding and adding missing atoms, so that loading PDB files will be faster
def fixPDB(pdb_file):
    """Uses the pdbfixer library to fix a pdb file, replacing non standard residues, removing
    hetero-atoms and adding missing hydrogens. The input is a pdb file location,
    the output is a fixer object, which is a pdb in the openawsem format."""
    fixer = pdbfixer.PDBFixer(filename=pdb_file, )
    fixer.findMissingResidues()
    chains = list(fixer.topology.chains())
    keys = fixer.missingResidues.keys()
    for key in list(keys):
        chain_tmp = chains[key[0]]
        if key[1] in [0, len(list(chain_tmp.residues()))]:
            del fixer.missingResidues[key]

    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(keepWater=False)
    # fixPDB will be accelerated if not perform finding and adding missing atoms
    #fixer.findMissingAtoms()
    #fixer.addMissingAtoms()
    #fixer.addMissingHydrogens(7.0)
    return fixer

In [3]:
# load all-atom chromatin pdb file including histones and DNA
#fix = open3SPN2.fixPDB('/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/6merRenumberResidues.pdb') # open3SPN2.fixPDB is slower 
fix = fixPDB('/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/6merRenumberResidues.pdb')
complex_table = open3SPN2.pdb2table(fix) # complex_table includes the information of all the atoms
print(complex_table)

       recname  serial name altLoc resname chainID  resSeq iCode       x  \
serial                                                                     
1         ATOM       1    N            ALA       A       1       -25.122   
2         ATOM       2   CA            ALA       A       1       -26.605   
3         ATOM       3    C            ALA       A       1       -27.034   
4         ATOM       4    O            ALA       A       1       -27.415   
5         ATOM       5   CB            ALA       A       1       -27.146   
...        ...     ...  ...    ...     ...     ...     ...   ...     ...   
40258     ATOM   40258   C4             DT       J    7808        59.060   
40259     ATOM   40259   O4             DT       J    7808        58.591   
40260     ATOM   40260   C5             DT       J    7808        58.967   
40261     ATOM   40261   C7             DT       J    7808        58.251   
40262     ATOM   40262   C6             DT       J    7808        59.528   

           

In [7]:
# pick out the heavy atoms from all the histones and run smog
# Q: How are the heavy atoms picked? 

# use /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/heavy.pdb

smog_main_path = '/Users/administrator/Documents/Tools/smog-2.2'
smog_file_dir_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files_smliu/chromatin-6mer/smog'
if not os.path.exists(smog_file_dir_path):
    os.makedirs(smog_file_dir_path)

pdb_src_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/heavy.pdb'
pdb_tgt_path = '%s/histone_atoms.pdb' % smog_file_dir_path
cmd = 'cp %s %s; ' % (pdb_src_path, pdb_tgt_path)
cmd = cmd + 'source %s/configure.smog2; ' % smog_main_path
cmd = cmd + 'cd %s; ' % smog_file_dir_path
sbm_aa_path = '%s/share/templates/SBM_AA' % smog_main_path
sbm_calpha_gaussian_path = '%s/share/templates/SBM_calpha+gaussian' % smog_main_path
cmd = cmd + 'smog2 -i histone_atoms.pdb -t %s -tCG %s' % (sbm_aa_path, sbm_calpha_gaussian_path)
print(cmd)
os.system(cmd)


cp /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/heavy.pdb /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files_smliu/chromatin-6mer/smog/histone_atoms.pdb; source /Users/administrator/Documents/Tools/smog-2.2/configure.smog2; cd /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files_smliu/chromatin-6mer/smog; smog2 -i histone_atoms.pdb -t /Users/administrator/Documents/Tools/smog-2.2/share/templates/SBM_AA -tCG /Users/administrator/Documents/Tools/smog-2.2/share/templates/SBM_calpha+gaussian
*****************************************************************************************

******* ******* ******* ******* ******* SMOG v2.2 ******* ******* ******* ******* *******
                                            
              Thank you for using the Structure-based Model (SMOG) software
                                            
    This package is the product of contributions from a number of people, including:
       


WARNING 1 : Angle between atoms 5238 5239 5240 (PDB lines 5243 5244 5245) greater than largeAngles (see .sif): theta=151.13 degrees
	Be careful!  This can lead to numerical instability of your simulations.


WARNING 2 : Angle between atoms 5257 5258 5259 (PDB lines 5262 5263 5264) greater than largeAngles (see .sif): theta=153.56 degrees
	Be careful!  This can lead to numerical instability of your simulations.


WARNING 3 : Angle between atoms 5240 5241 5242 (PDB lines 5245 5246 5247) greater than largeAngles (see .sif): theta=166.23 degrees
	Be careful!  This can lead to numerical instability of your simulations.


WARNING 4 : Angle between atoms 3292 3293 3294 (PDB lines 3295 3296 3297) greater than largeAngles (see .sif): theta=166.23 degrees
	Be careful!  This can lead to numerical instability of your simulations.


WARNING 5 : Angle between atoms 3290 3291 3292 (PDB lines 3293 3294 3295) greater than largeAngles (see .sif): theta=151.11 degrees
	Be careful!  This can lead to nume

Saving dihedrals
NOTE: Total number of atoms included in normalization is 45468, and 5844 atoms are in the system.
Completing final details...

Your Structure-based Model is ready!

Files generated: 
	smog.top
	smog.gro
	smog.ndx
	smog.contacts
	smog.contacts.CG

########################################################################################

              We kindly ask that you cite the appropriate primary references
             when using SMOG models. See http://www.smog-server.org/refs.html

--Software-- 
Noel JK, Levi M, Raghunathan M, Lammert H, Hayes R, Onuchic JN, and Whitford PC. (2016)
SMOG V2, A Versatile Software Package for Generating Structure-Based Models.
PLoS Comput Biol 12(3): e1004794. doi:10.1371/journal.pcbi.1004794

--C-alpha Model-- 
Clementi C, Nymeyer H, Onuchic JN (2000) Journal of Molecular Biology 298, 937-953. 

--Contact Potentials--
Shadow Algorithm: Noel JK, Whitford PC, Onuchic, JN (2012) J Phys Chem, 116, 8692-8702.
C-alpha Gaussian Interactio

0

In [9]:
# pick out sections from smog.top
smog_file_dir_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files_smliu/chromatin-6mer/smog'
cmd = 'cd %s; ' % smog_file_dir_path
py_get_section_script_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/getSection.py'
key_word_list = ['atoms', 'bonds', 'angles', 'dihedrals', 'pairs', 'exclusions', 'system']
for i in range(len(key_word_list) - 1):
    key_word_1 = key_word_list[i]
    key_word_2 = key_word_list[i + 1]
    cmd = cmd + 'python %s ./smog.top %s.dat "[ %s ]" "[ %s ]"; ' % (py_get_section_script_path, key_word_1, key_word_1, key_word_2)
print(cmd)
os.system(cmd)
    

cd /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files_smliu/chromatin-6mer/smog; python /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/getSection.py ./smog.top atoms.dat "[ atoms ]" "[ bonds ]"; python /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/getSection.py ./smog.top bonds.dat "[ bonds ]" "[ angles ]"; python /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/getSection.py ./smog.top angles.dat "[ angles ]" "[ dihedrals ]"; python /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/getSection.py ./smog.top dihedrals.dat "[ dihedrals ]" "[ pairs ]"; python /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/getSection.py ./smog.top pairs.dat "[ pairs ]" "[ exclusions ]"; python /Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/6mer/NRL167/getSection.py ./smog.top exclusions.dat "[ 

0

In [7]:
# generate DNA CG model from complex_table
#print(complex_table)
dna_cg_atoms = open3SPN2.DNA.CoarseGrain(complex_table)
print(dna_cg_atoms[['serial', 'name', 'resname']])

protein_cg_atoms = ffCalpha.Protein.CoarseGrain(complex_table)
#print(protein_cg_atoms[:3])
print(protein_cg_atoms[['serial', 'name', 'resname']])



      serial name resname
0          0    A      DA
1          1    S      DA
2          2    C      DC
3          3    P      DC
4          4    S      DC
...      ...  ...     ...
5885    5885    P      DG
5886    5886    S      DG
5887    5887    T      DT
5888    5888    P      DT
5889    5889    S      DT

[5890 rows x 3 columns]
      serial name resname
0          0   CA     ALA
1          1   CA     ARG
2          2   CA     THR
3          3   CA     LYS
4          4   CA     GLN
...      ...  ...     ...
5839    5839   CA     TYR
5840    5840   CA     THR
5841    5841   CA     SER
5842    5842   CA     ALA
5843    5843   CA     LYS

[5844 rows x 3 columns]


In [13]:
# merge CG protein and dna
cg_fiber = pd.concat([protein_cg_atoms, dna_cg_atoms], sort=False)
cg_fiber.index = range(len(cg_fiber))
cg_fiber['serial'] = list(cg_fiber.index)

print(cg_fiber[['serial', 'name', 'resname']])

# write the merged CG structure
main_dir_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files_smliu/chromatin-6mer/cg-fiber'
if not os.path.exists(main_dir_path):
    os.makedirs(main_dir_path)

ffCalpha.writePDB(cg_fiber, '%s/cg_fiber.pdb' % main_dir_path)
ffCalpha.save_protein_sequence(cg_fiber, sequence_file='%s/protein_seq.txt' % main_dir_path)

       serial name resname
0           0   CA     ALA
1           1   CA     ARG
2           2   CA     THR
3           3   CA     LYS
4           4   CA     GLN
...       ...  ...     ...
11729   11729    P      DG
11730   11730    S      DG
11731   11731    T      DT
11732   11732    P      DT
11733   11733    S      DT

[11734 rows x 3 columns]


# Build the simulation

In [9]:
# start to perform simulations with openmm
# set some paths
main_dir_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu/chromatin-6mer/cg-fiber' # path on desktop
#main_dir_path = '/Users/smliu/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu/chromatin-6mer/cg-fiber' # path on laptop
ffcalpha_xml_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/ffCalpha.xml' # path on desktop
#ffcalpha_xml_path = '/Users/smliu/Documents/Projects/CA_SBM_3SPN2C_OPENMM/calphaSBM/ffCalpha.xml' # path on laptop

os.chdir(main_dir_path) # write the output files to main_dir_path
pdb = simtk.openmm.app.PDBFile('%s/cg_fiber.pdb' % main_dir_path)
top = pdb.topology
coord = pdb.positions

forcefield = simtk.openmm.app.ForceField(ffcalpha_xml_path, open3SPN2.xml)
s = forcefield.createSystem(top)

# create the DNA and protein objects
dna = open3SPN2.DNA.fromCoarsePDB('%s/cg_fiber.pdb' % main_dir_path, dna_type='B_curved')
dna.computeTopology(template_from_X3DNA=True) # create bond, angle, and dihedral tables with parameters, and the parameters are chosen by X3DNA
with open('%s/protein_seq.txt' % main_dir_path) as ps:
    protein_seq = ps.readlines()[0]
protein = ffCalpha.Protein.fromCoarsePDB('%s/cg_fiber.pdb' % main_dir_path, sequence=protein_seq)

dna.periodic = False
protein.periodic = False


Time used: 00:00:00:00


Empty DataFrame
Columns: [recname_old, serial_old, name_old, altLoc_old, resname_old, chainID_old, resSeq_old, iCode_old, x_old, y_old, z_old, occupancy_old, tempFactor_old, element_old, charge_old, type_old, recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []

[0 rows x 32 columns]



Time used: 00:00:00:01


Empty DataFrame
Columns: [recname_old, serial_old, name_old, altLoc_old, resname_old, chainID_old, resSeq_old, iCode_old, x_old, y_old, z_old, occupancy_old, tempFactor_old, element_old, charge_old, type_old, recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []

[0 rows x 32 columns]
[5004, 5106, 5234, 5356, 5491, 5593, 5721, 5843]


In [10]:
print(protein)
print(dna)

3SPN2 DNA object (11734 atoms)


In [4]:
# produce globular domain index file for a single histone
# note in openmm atom index starts from 0
histone_tail = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 
                        39, 40, 41, 42, 43, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 
                        146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 
                        238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 
                        252, 253, 254, 255, 256, 257, 353, 354, 355, 356, 357, 358, 359, 360, 
                        361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 
                        375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 
                        389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 488, 489, 
                        490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 
                        504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 
                        518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 623, 
                        624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 
                        638, 639, 640, 641, 642, 643, 644, 645, 646, 725, 726, 727, 728, 729, 
                        730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 
                        744, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 
                        853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 
                        867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 
                        881, 882, 883, 884, 885, 886, 887], dtype=int) - 1

n_CA_each_nucl = 974
histone_core = []
for each in range(n_CA_each_nucl):
    if each not in histone_tail:
        histone_core.append(each)
histone_core = np.array(histone_core, dtype=int)

# write histone tail atom index file
#files_smliu_path = '/Users/smliu/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu' # path on laptop
files_smliu_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu' # path on desktop
with open('%s/single_nucl_tail_index.txt' % files_smliu_path, 'w') as output_file:
    for each in histone_tail:
        output_file.write('%d ' % each)
    output_file.write('\n')

# write histone core atom index file
#files_smliu_path = '/Users/smliu/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu' # path on laptop
files_smliu_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu' # path on desktop
with open('%s/single_nucl_core_index.txt' % files_smliu_path, 'w') as output_file:
    for each in histone_core:
        output_file.write('%d ' % each)
    output_file.write('\n')



In [5]:
# get 2d list that each sublist includes the atom id for the corresponding chain
# atom index starts from 0

# for a single nucleosome
n_CA_each_chain_in_each_nucl = [135, 102, 128, 122, 135, 102, 128, 122]
cumsum_n_CA_each_chain_in_each_nucl = np.cumsum(np.array(n_CA_each_chain_in_each_nucl, dtype=int))
histone_chains = []
n_chains_each_histone = len(n_CA_each_chain_in_each_nucl)
for i in range(n_chains_each_histone):
    histone_chains.append([])

atom_id = 0
chain_id = 0
n_CA_each_nucl = 974
while atom_id < n_CA_each_nucl:
    histone_chains[chain_id].append(atom_id)
    atom_id += 1
    if atom_id >= cumsum_n_CA_each_chain_in_each_nucl[chain_id]:
        chain_id += 1

# for all the histones
histones_rigid_identity = []
n_nucl = 6
for i in range(n_nucl*n_CA_each_nucl):
    if (i % n_CA_each_nucl) in histone_tail:
        histones_rigid_identity.append(int(i/n_CA_each_nucl))
    else:
        histones_rigid_identity.append(None)
# histones_rigid_identity[i] shows which rigid body the i-th atom belongs to, and if i-th atom does not belong to any rigid body, then histones_rigid_identity[i] is None

histones_chains = []
for i in range(n_nucl):
    for j in range(n_chains_each_histone):
        chain = np.array(histone_chains[j], dtype=int) + i*n_CA_each_nucl
        histones_chains.append(chain.tolist())

# set rigid identity for the whole fiber
nrl = 167





In [6]:
scale_factor = 2.5 # scale factor for all the SBM related potentials

# set force dictionary
forces = {}

smog_file_dir_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu/chromatin-6mer/smog' # path on desktop
main_dir_path = '/Users/administrator/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu/chromatin-6mer/cg-fiber' # path on desktop

#smog_file_dir_path = '/Users/smliu/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu/chromatin-6mer/smog' # path on laptop
#main_dir_path = '/Users/smliu/Documents/Projects/CA_SBM_3SPN2C_OPENMM/files-smliu/chromatin-6mer/cg-fiber' # path on laptop

smog_atoms_file_path = '%s/atoms.dat' % smog_file_dir_path
smog_bonds_file_path = '%s/bonds.dat' % smog_file_dir_path
smog_angles_file_path = '%s/angles.dat' % smog_file_dir_path
smog_dihedrals_file_path = '%s/dihedrals.dat' % smog_file_dir_path
smog_exclusions_file_path = '%s/exclusions.dat' % smog_file_dir_path
smog_pairs_file_path = '%s/pairs.dat' % smog_file_dir_path

exclusions_list = ffCalphaExt.buildNonBondedExclusionsList(histones_chains, smog_exclusions_file_path, histones_rigid_identity)

for force_name in open3SPN2.forces:
    print('force_name = %s' % force_name)
    #print(open3SPN2.forces[force_name])
    force = open3SPN2.forces[force_name](dna) # object of different types of 3SPN2 force on the input dna
    if force_name in ['BasePair', 'CrossStacking']:
        force.addForce(s)
        # note usually addForce is a method for the system, but addForce method is defined for BasePair and CrossStacking classes
    elif force_name in ['Exclusion']:
        #ffCalpha.addNonBondedExclusions(protein, force, "bonds.dat", "angles.dat", "dihedrals.dat", "exclusions.dat")
        for each in exclusions_list:
            force.addExclusion(each[0], each[1])
        print("Number of exclusions is: ", force.getNumExclusions())
        s.addForce(force)
    elif force_name in ['Electrostatics']:
        #ffCalpha.addNonBondedExclusions(protein, force, "bonds.dat", "angles.dat", "dihedrals.dat", "exclusions.dat")
        for each in exclusions_list:
            force.addExclusion(each[0], each[1])
        print("Number of exclusions is: ", force.getNumExclusions())
        s.addForce(force)
    else:
        s.addForce(force)

    forces.update({force_name:force})


force_name = Bond
force_name = Angle
force_name = Stacking
force_name = Dihedral
force_name = BasePair
force_name = CrossStacking
force_name = Exclusion
Number of exclusions is:  525760
force_name = Electrostatics
0.7853757564223662 nm 2.3489471160081776 mol/(nm kcal)
Number of exclusions is:  525760


In [7]:
# Add DNA-protein interaction forces
for force_name in open3SPN2.protein_dna_forces:
    print('force_name = %s' % force_name)
    force = open3SPN2.protein_dna_forces[force_name](dna, protein)
    if force_name in ['ExclusionProteinDNA']:    
        #ffCalpha.addNonBondedExclusions(protein, force, "bonds.dat", "angles.dat", "dihedrals.dat", "exclusions.dat")
        for each in exclusions_list:
            force.addExclusion(each[0], each[1])
        print("Number of exclusions is: ", force.getNumExclusions())
    elif force_name in ['ElectrostaticsProteinDNA']:
        #ffCalpha.addNonBondedExclusions(protein, force, "bonds.dat", "angles.dat", "dihedrals.dat", "exclusions.dat")
        for each in exclusions_list:
            force.addExclusion(each[0], each[1])
        print("Number of exclusions is: ", force.getNumExclusions()) 
    else:
        print("Number of exclusions is: ", force.getNumExclusions()) 
    
    s.addForce(force)
    forces.update({force_name: force})


force_name = ExclusionProteinDNA
Number of exclusions is:  525760
force_name = ElectrostaticsProteinDNA
7.853757564223662e-09 dm 2.3489471160081776 mol/(nm kcal)
Number of exclusions is:  525760


In [8]:
# Add Calpha forces
openCalphaforces = dict(SBM_Bond=ffCalpha.functionTerms.basicTermsExt.bond_term,
                        SBM_Angle=ffCalpha.functionTerms.basicTermsExt.angle_term,
                        SBM_Dihedral=ffCalpha.functionTerms.basicTermsExt.dihedral_term,
                        SBM_Pair=ffCalpha.functionTerms.basicTermsExt.pair_term,
                        SBM_Nonbonded=ffCalpha.functionTerms.basicTermsExt.nonbonded_term,
                        SBM_Electrostatics=ffCalpha.functionTerms.basicTermsExt.debye_huckel_term,
                        )

